<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_LSTM%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%83%9D%EC%84%B1AI_%EB%AA%A8%EB%8D%B8%EB%A7%81_pytorch_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 데이터 불러오기

### 전체 데이터셋

In [ ]:
pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean_remove.csv").columns[:3]

Index(['review', 'reply', 'star'], dtype='object')

In [ ]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean_remove.csv")
cols = train.columns[:3] # 사용할 컬럼(review, reply, star)만 가져오기
train = train[cols]
train.isnull().sum()

review     0
reply     25
star       0
dtype: int64

In [ ]:
idx_null = train[train['reply'].isnull()].index
train = train.drop(idx_null, axis=0)
train.isnull().sum()

review    0
reply     0
star      0
dtype: int64

In [ ]:
train['star'].value_counts().sort_index() # 별점 분포 확인

1      111
2       87
3      226
4      770
5    23779
Name: star, dtype: int64

In [ ]:
print(len((train[train['star'] != 5])))
len(train[train['star'] != 5])/len(train[train['star'] == 5])

1194


0.05021237226123891

### 오버샘플링(paraphrase 데이터)

In [ ]:
train_over = pd.read_csv(f"{DATA_PATH}train_over_0905.csv")
train_over.isnull().sum()

review    0
reply     2
star      0
dtype: int64

In [ ]:
idx_null_over = train_over[train_over['reply'].isnull()].index
train_over = train_over.drop(idx_null_over, axis=0)
train_over.isnull().sum()

review    0
reply     0
star      0
dtype: int64

In [ ]:
train = pd.concat([train, train_over], axis=0).reset_index(drop=True)
train

,review,reply,star
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제...,5
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...,5
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,안녕하세요 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기 망...,5
3,맛있습니다 근데 국물이 안와서 아쉽습니다,안녕하세요 저희 측 실수로 인하여 누락이 되었군요 이에 대해서 사과 말씀 드립니다 ...,4
4,맛있게잘먹었습니다,헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요,5
...,...,...,...
26162,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,맛있는 식사가 되셨기를 바랍니다 맛있는 식사 이용해 주셔서 감사합니다,4
26163,맛있게 잘 먹었습니다 맛있게,저희는 배달 대행 업체인데 배달 중에 오류가 난 부분이 있으면 언제든지 연락주시면 ...,4
26164,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4
26165,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해본 결과 직...,4


### 여기까지 저장하기

In [ ]:
# train.to_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean_remove_over.csv", index=False)

# 여기까지 완료된 데이터 불러오기

In [ ]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean_remove_over.csv")

train['review'] = train['review'].astype(str)
train['reply'] = train['reply'].astype(str)
train

,review,reply,star
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제...,5
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...,5
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,안녕하세요 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기 망...,5
3,맛있습니다 근데 국물이 안와서 아쉽습니다,안녕하세요 저희 측 실수로 인하여 누락이 되었군요 이에 대해서 사과 말씀 드립니다 ...,4
4,맛있게잘먹었습니다,헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요,5
...,...,...,...
26162,맛있어요 양도 많고 푸짐하고 아주 맛있어요 맛도 있고 아주 만족스러운 식사였습니다,맛있는 식사가 되셨기를 바랍니다 맛있는 식사 이용해 주셔서 감사합니다,4
26163,맛있게 잘 먹었습니다 맛있게,저희는 배달 대행 업체인데 배달 중에 오류가 난 부분이 있으면 언제든지 연락주시면 ...,4
26164,맛있게 잘 먹었어요 맛있게 잘,항상 좋은 음식으로 보답하는 것으로 보답하겠습니다 이용해 주셔서 감사합니다 더불어 ...,4
26165,괜찮네요 괜찮은 것 같아요 맛도 괜찮고 가격도 괜찮고,아무래도 매장에서 드시는 것보다는 조금 미흡할 수 있으나 여러모로 시행해본 결과 직...,4


In [ ]:
train.isnull().sum()

review    0
reply     0
star      0
dtype: int64

- 개별 삭제(추론 결과보고 계속 추가)

In [ ]:
print(train[train['reply'].str.contains("민의")].index)

Int64Index([], dtype='int64')


In [ ]:
remove_keywords = ['북서울꿈의숲','민의']
for keyword in remove_keywords:
    train['reply'] = train['reply'].str.replace(keyword, '')

train['reply'][6822]

'고객님 안녕하세요 항상 고객님께 최선을 다하는 뜸들이다  소중한 별5 리뷰 감사합니다 맛있게 드셔 주셔서 정말 행복해요 다음에도 만족하실 수 있도록 최선을 다하겠습니다 감사합니다 뜸들이다는  이 최고 영암군에서 재배한 햅쌀만을 사용합니다 밥을 소량씩 자주 지어 갓 지은 밥 맛을 추구합니다 매장에서 직접 아보카도를 후숙하여 더욱더 고소합니다  주변에서 직접 홀 매장 영업하고 있습니다 당일재료 당일소진 모든 재료가 소진되면 영업을 종료합니다'

In [ ]:
train['star'].value_counts().sort_index() # 별점 분포 확인

1      222
2      174
3      453
4     1539
5    23779
Name: star, dtype: int64

In [ ]:
train_1 = train[train['star'] == 1]
train_2 = train[train['star'] == 2]
train_3 = train[train['star'] == 3]
train_4 = train[train['star'] == 4]

# 5점 데이터 중 중복된 답변제거
train_5 = train[train['star'] == 5].drop_duplicates(subset='reply', keep=False)
train_5['star'].value_counts().sort_index()

# train_star = pd.concat([train_1, train_2, train_3, train_4], ignore_index=True)
# train_star

5    10857
Name: star, dtype: int64

In [ ]:
length = len(train_1+train_2+train_3+train_4)
length

2388

In [ ]:
shuffled_train_5 = train_5.sample(frac=1, random_state=42)
# train_5 = shuffled_train_5.iloc[:length].reset_index(drop=True) # 1:1 비율
train_5 = shuffled_train_5.iloc[:length*2].reset_index(drop=True) # 2:1 비율 만들기

train = pd.concat([train_1, train_2, train_3, train_4, train_5], ignore_index=True)
train['star'].value_counts().sort_index()

1     222
2     174
3     453
4    1539
5    4776
Name: star, dtype: int64

In [ ]:
# 다시 섞기
train = train.sample(frac=1, random_state=42).reset_index(drop=True)
train

,review,reply,star
0,저는 개인적으로 생각보다 막국수 양이 너무 많아서 한 번도 못 먹고 버린 게 더 많...,고객님께서 요청하시는 사항은 저희가 빠뜨리지 않고 챙겨드리는데 저녁시간에 조금 밀리...,4
1,닭발이 전분끼 있는 것 처럼 끈적거려요,전분은 들어가지않습니다 배달이 늦은 탓일까요 죄송합니다 다음에는 더욱 만족드릴수있도...,3
2,오우 재료도 신선하고 가성비 좋은것 같아요 서비스로 주시는 쥬스도 맛있어요,안녕하세요 항상 신선한 재료로 맛있는 전달드릴려구 노력하고있습니다 맛과 양 영양까지...,5
3,맛있게 잘먹었습니다,안녕하세요 를 찾아주셔서 진심으로 감사합니다 맛있게 잘 드셨다니 넘 행복합니당 즐거...,5
4,좋아요,감사합니다 언제나 맛으로보답드릴 수 있는,5
...,...,...,...
7159,여행와서 맛있게 먹고 갑니다,저희 찾아주셔서 감사드립니다 여행오셔서 맛있게 드셨다니 너무감사드립니다 앞으로도 저...,5
7160,맛이 자극적이지 않고 맛있어요 맛좋고 정갈한 반찬 잘먹겠습니다,주문 감사드립니다 짜거나 맵고 싱겁게 만들지 않도록 최대한 노력하고 있답니다 맛있게...,5
7161,정말 푸짐하게 나옵니다,리뷰 감사합니다 푸짐하게 드셨다니 기분 좋네요 다음에도 많이많이 이용해주세요,5
7162,배달도 빠르고 정말 맛있게 먹었습니다,먼저 저희 매장을 이용해 주셔서 감사합니다 사진리뷰와 별5개 너무나 감사드립니다 배...,5


In [ ]:
review, reply = train["review"] , train["reply"]
len(review) , len(reply)

(7164, 7164)

# 한국어 형태소 분석

In [ ]:
review[:5]

0    저는 개인적으로 생각보다 막국수 양이 너무 많아서 한 번도 못 먹고 버린 게 더 많...
1                                닭발이 전분끼 있는 것 처럼 끈적거려요
2           오우 재료도 신선하고 가성비 좋은것 같아요 서비스로 주시는 쥬스도 맛있어요 
3                                          맛있게 잘먹었습니다 
4                                                  좋아요
Name: review, dtype: object

In [ ]:
import unicodedata

def normalize(sentence):
    return unicodedata.normalize('NFC', sentence)

normalize(review[0])

'저는 개인적으로 생각보다 막국수 양이 너무 많아서 한 번도 못 먹고 버린 게 더 많을 정도로 별로였어요.'

# 단어 사전 만들기

In [ ]:
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2

class WordVocab():
    def __init__(self):
        self.word2index = {'<PAD>': PAD_TOKEN,
                           '<SOS>': SOS_TOKEN,
                           '<EOS>': EOS_TOKEN,}
        self.word2count = {}
        self.index2word = {PAD_TOKEN: '<PAD>',
                           SOS_TOKEN: '<SOS>',
                           EOS_TOKEN: '<EOS>'}
        self.n_words = 3  # PAD, SOS, EOS 포함

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
print(f'원문: {review[0]}')
lang = WordVocab()
lang.add_sentence(review[0])
print('단어사전: ', lang.word2index)

원문: 저는 개인적으로 생각보다 막국수 양이 너무 많아서 한 번도 못 먹고 버린 게 더 많을 정도로 별로였어요.
단어사전:  {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '저는': 3, '개인적으로': 4, '생각보다': 5, '막국수': 6, '양이': 7, '너무': 8, '많아서': 9, '한': 10, '번도': 11, '못': 12, '먹고': 13, '버린': 14, '게': 15, '더': 16, '많을': 17, '정도로': 18, '별로였어요.': 19}


# 데이터셋 만들기

In [ ]:
from konlpy.tag import Okt

In [ ]:
class TextDataset(Dataset):
    def __init__(self, df, min_length=3, max_length=32):
        super().__init__()

        # TOKEN 정의
        self.PAD_TOKEN = 0 # Padding 토큰
        self.SOS_TOKEN = 1 # SOS 토큰
        self.EOS_TOKEN = 2 # EOS 토큰

        self.tagger = Okt()   # 형태소 분석기
        self.max_length = max_length # 한 문장의 최대 길이 지정

        # src: 리뷰, tgt: 답변
        src_clean = []
        tgt_clean = []
        store_list = []

        # 단어 사전 생성
        wordvocab = WordVocab()

        for _, row in df.iterrows():
            src = row['review']
            tgt = row['reply']

            # 한글 전처리
            src = self.clean_text(src)
            tgt = self.clean_text(tgt)

            if len(src.split()) > min_length and len(tgt.split()) > min_length:
                # 최소 길이를 넘어가는 문장의 단어만 추가
                wordvocab.add_sentence(src)
                wordvocab.add_sentence(tgt)
                src_clean.append(src)
                tgt_clean.append(tgt)

        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocab = wordvocab

    def clean_text(self, sentence):
        # 형태소 처리
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence

    def texts_to_sequences(self, sentence):
        # 문장 -> 시퀀스로 변환
        return [self.wordvocab.word2index[w] for w in sentence.split()]

    def pad_sequence(self, sentence_tokens):
        # 문장의 맨 끝 토큰은 제거
        sentence_tokens = sentence_tokens[:(self.max_length-1)]
        token_length = len(sentence_tokens)

        # 문장의 맨 끝부분에 <EOS> 토큰 추가
        sentence_tokens.append(self.EOS_TOKEN)

        for i in range(token_length, (self.max_length-1)):
            # 나머지 빈 곳에 <PAD> 토큰 추가
            sentence_tokens.append(self.PAD_TOKEN)
        return sentence_tokens

    def __getitem__(self, idx):
        inputs = self.srcs[idx]
        inputs_sequences = self.texts_to_sequences(inputs)
        inputs_padded = self.pad_sequence(inputs_sequences)

        outputs = self.tgts[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)

        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)

    def __len__(self):
        return len(self.srcs)

In [ ]:
MAX_LENGTH = 32
dataset = TextDataset(train, min_length=3, max_length=MAX_LENGTH)

## 데이터셋 분할(테스트 데이터 생성)

In [ ]:
# split_ratio = 0.2  # 테스트 데이터의 비율(20%)
# split_index = int(len(train) * split_ratio)

# test_data = train[:split_index] # 증강데이터가 없는 앞에서부터 20% 할당
# train_data = train[split_index:]

# len(test_data) , len(train_data)

(3882, 15530)

In [ ]:
# train_dataset = TextDataset(train_data, min_length=3, max_length=MAX_LENGTH)
# test_dataset = TextDataset(test_data, min_length=3, max_length=MAX_LENGTH)

In [ ]:
x, y = dataset[0]

In [ ]:
print(f'x shape: {x.shape}')
print(x)

x shape: torch.Size([32])
tensor([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28,  2,  0,  0,  0,  0,  0])


In [ ]:
print(f'y shape: {y.shape}')
print(y)

y shape: torch.Size([32])
tensor([29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
        47, 48, 49, 50, 51, 52, 53, 54, 55, 50, 56, 28,  2,  0])


In [ ]:
from torch.utils.data import random_split

train_size = int(len(dataset) * 0.8) # 80%의 데이터를 train에 할당
test_size = len(dataset) - train_size # 나머지 20% 데이터를 test에 할당
print(train_size, test_size)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size]) # 랜덤 스플릿으로 분할

4932 1233


# 데이터로더 만들기

In [ ]:
from torch.utils.data import DataLoader, SubsetRandomSampler

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# 1개의 배치 데이터를 추출
x, y = next(iter(train_loader))

# shape: (batch_size, sequence_length)
x.shape, y.shape

(torch.Size([16, 32]), torch.Size([16, 32]))

# Encoder 만들기

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        super().__init__()

        # 단어 사전의 개수 지정
        self.num_vocabs = num_vocabs
        # 임베딩 레이어 정의 (number of vocabs, embedding dimension)
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        # GRU (embedding dimension)
        self.gru = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)
        output, hidden = self.gru(x)
        return output, hidden

# Decoder 만들기

In [ ]:
class Decoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers=1, dropout=0.2):
        super().__init__()
        # 단어사전 개수
        self.num_vocabs = num_vocabs
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

        # 최종 출력은 단어사전의 개수
        self.fc = nn.Linear(hidden_size, num_vocabs)

    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) # (1, batch_size) 로 변환
        embedded = F.relu(self.embedding(x))
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden_state)
        output = self.fc(output.squeeze(0)) # (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
        return output, hidden

# Seq2Seq 모델 만들기

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        # inputs : (batch_size, sequence_length)
        # outputs: (batch_size, sequence_length)

        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_vocabs

        # 리턴할 예측된 outputs를 저장할 임시 변수
        # (sequence_length, batch_size, num_vocabs)
        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)

        # 인코더에 입력 데이터 주입, encoder_output은 버리고 hidden_state 만 살립니다.
        # 여기서 hidden_state가 디코더에 주입할 context vector 입니다.
        # (Bidirectional(1) x number of layers(1), batch_size, hidden_size)
        _, decoder_hidden = self.encoder(inputs)

        # (batch_size) shape의 SOS TOKEN으로 채워진 디코더 입력 생성
        decoder_input = torch.full((batch_size,), SOS_TOKEN, device=self.device)

        # 순회하면서 출력 단어를 생성합니다.
        # 0번째는 SOS TOKEN이 위치하므로, 1번째 인덱스부터 순회합니다.
        for t in range(0, output_length):
            # decoder_input : 디코더 입력 (batch_size) 형태의 SOS TOKEN로 채워진 입력
            # decoder_output: (batch_size, num_vocabs)
            # decoder_hidden: (Bidirectional(1) x number of layers(1), batch_size, hidden_size), context vector와 동일 shape
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)

            # t번째 단어에 디코더의 output 저장
            predicted_outputs[t] = decoder_output

            # teacher forcing 적용 여부 확률로 결정
            # teacher forcing 이란: 정답치를 다음 RNN Cell의 입력으로 넣어주는 경우. 수렴속도가 빠를 수 있으나, 불안정할 수 있음
            teacher_force = random.random() < teacher_forcing_ratio

            # top1 단어 토큰 예측
            top1 = decoder_output.argmax(1)

            # teacher forcing 인 경우 ground truth 값을, 그렇지 않은 경우, 예측 값을 다음 input으로 지정
            decoder_input = outputs[:, t] if teacher_force else top1

        return predicted_outputs.permute(1, 0, 2) # (batch_size, sequence_length, num_vocabs)로 변경

In [ ]:
# Encoder 정의
encoder = Encoder(num_vocabs=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Seq2Seq 정의
seq2seq = Seq2Seq(encoder, decoder, 'cpu')

In [ ]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)
# (batch_size, sequence_length), (batch_size, sequence_length)

torch.Size([16, 32]) torch.Size([16, 32])


In [ ]:
output = seq2seq(x, y)
print(output.shape)
# (batch_size, sequence_length, num_vocabs)

torch.Size([16, 32, 18747])


## 모델 확인

In [ ]:
NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_SIZE = 512
EMBEDDIMG_DIM = 256

print(f'num_vocabs: {NUM_VOCABS}\n======================')

# Encoder 정의
encoder = Encoder(num_vocabs=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)

# Seq2Seq 생성
# encoder, decoder를 device 모두 지정
model = Seq2Seq(encoder.to(device), decoder.to(device), device)
print(model)

num_vocabs: 18747
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(18747, 256)
    (gru): GRU(256, 512)
  )
  (decoder): Decoder(
    (embedding): Embedding(18747, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (gru): GRU(256, 512)
    (fc): Linear(in_features=512, out_features=18747, bias=True)
  )
)


# 하이퍼파라미터 정의

In [ ]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0

        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta


    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')

        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')


        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [ ]:
LR = 1e-3
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
es = EarlyStopping(patience=5, delta=0.001, mode='min', verbose=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

# 학습 loop 만들기

In [ ]:
def train(model, data_loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0

    for x, y in data_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        # output: (batch_size, sequence_length, num_vocabs)
        output = model(x, y)
        output_dim = output.size(2)

        # 1번 index 부터 슬라이싱한 이유는 0번 index가 SOS TOKEN 이기 때문
        # (batch_size*sequence_length, num_vocabs) 로 변경
        output = output.reshape(-1, output_dim)

        # (batch_size*sequence_length) 로 변경
        y = y.view(-1)

        # Loss 계산
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

    return running_loss / len(data_loader)

In [ ]:
def evaluate(model, data_loader, loss_fn, device):
    model.eval()

    eval_loss = 0

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)

            # Loss 계산
            loss = loss_fn(output, y)

            eval_loss += loss.item() * x.size(0)

    return eval_loss / len(data_loader)

# 추론 함수

In [ ]:
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS_TOKEN, EOS_TOKEN, PAD_TOKEN]:
            outputs.append(word)
        if word == EOS_TOKEN:
            break
    return ' '.join(outputs)

In [ ]:
!git clone https://github.com/ssut/py-hanspell.git
!cd py-hanspell
!pip install py-hanspell/

fatal: destination path 'py-hanspell' already exists and is not an empty directory.
Processing ./py-hanspell
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=37be0956c271b3b723f3d6834fe55ea92b61e1c28a0e0b3d12dcb029c05d9f5e
  Stored in directory: /root/.cache/pip/wheels/a3/a7/16/8db596361f402638188412108152b8fbf6db76c898e00fe02d
Successfully built py-hanspell
  Attempting uninstall: py-hanspell
    Found existing installation: py-hanspell 1.1
    Uninstalling py-hanspell-1.1:
      Successfully uninstalled py-hanspell-1.1


In [ ]:
from hanspell import spell_checker
def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

# 테스트 loop 만들기
- 챗봇 함수화

In [ ]:
# 전체 예측값을 데이터프레임 형식에 담기
def random_evaluation(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            data = {'test_review': [], 'chatbot_reply': []}
            for i in range(n_samples):
                x_result = sequence_to_sentence(x[i], index2word)
                y_result = sequence_to_sentence(y[i], index2word)
                pred_result = sequence_to_sentence(preds[i].argmax(1), index2word)
                pred_result_correct = correct_spell(pred_result)
                data['test_review'].append(x_result)
                data['chatbot_reply'].append(pred_result_correct)

    df = pd.DataFrame(data)
    print(df)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(text, generated_text):
    y = [text]  # 실제 데이터 y
    X = generated_text  # 생성된 텍스트 X

    # BLEU 점수 계산
    bleu_score = sentence_bleu(y, X)
    return bleu_score

In [ ]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=128, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=1,
                          no_repeat_ngram_size=4,
                          top_k=5,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

In [ ]:
# 10개만 보기
def random_evaluation_10(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            for i in range(n):
                result_review = sequence_to_sentence(x[i], index2word)
                result_reply = sequence_to_sentence(y[i], index2word)
                result_pred = sequence_to_sentence(preds[i].argmax(1), index2word)
                print(f'리뷰    : {result_review}')
                print(f'답변    : {result_reply}')
                print(f'예측답변: {paraphrase_text(result_pred)}') # paraphrased text 반환
                print(f'BLEU점수: {calculate_bleu(result_reply, result_pred)}')
                print('==='*10)

# 학습하기

In [ ]:
# yogiyo_paraphrase
NUM_EPOCHS = 30
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905_over.pt'

best_loss = np.inf

for epoch in tqdm(range(NUM_EPOCHS), total=NUM_EPOCHS):
    loss = train(model, train_loader, optimizer, loss_fn, device)

    val_loss = evaluate(model, test_loader, loss_fn, device)

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_PATH)

    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')

    # Early Stop
    es(loss)
    if es.early_stop:
        break

    # Scheduler
    scheduler.step(val_loss)

model.load_state_dict(torch.load(STATEDICT_PATH))
torch.save(model.state_dict(), f'/content/drive/MyDrive/project_final/data/models/yogiyo_0905_over{best_loss:.4f}.pt')

  0%|          | 0/30 [00:00<?, ?it/s]

epoch: 1, loss: 86.8687, val_loss: 79.6212
[EarlyStopping] (Update) Best Score: 86.86869
[EarlyStopping] (Update) Best Score: 76.74496
[EarlyStopping] (Update) Best Score: 72.14059
[EarlyStopping] (Update) Best Score: 68.70583
[EarlyStopping] (Update) Best Score: 65.44263
epoch: 6, loss: 62.7642, val_loss: 70.1436
[EarlyStopping] (Update) Best Score: 62.76420
[EarlyStopping] (Update) Best Score: 60.37220
[EarlyStopping] (Update) Best Score: 57.57310
[EarlyStopping] (Update) Best Score: 54.66249
[EarlyStopping] (Update) Best Score: 52.92570
Epoch 00010: reducing learning rate of group 0 to 5.0000e-04.
epoch: 11, loss: 48.0427, val_loss: 70.6609
[EarlyStopping] (Update) Best Score: 48.04269
[EarlyStopping] (Update) Best Score: 46.41129
[EarlyStopping] (Update) Best Score: 44.26264
Epoch 00013: reducing learning rate of group 0 to 2.5000e-04.
[EarlyStopping] (Update) Best Score: 41.87723
[EarlyStopping] (Update) Best Score: 40.33631
epoch: 16, loss: 40.0528, val_loss: 73.2118
[EarlyStoppi

# 결과 확인하기

In [ ]:
# 0905 데이터 + over + 긍부정 비율 2:1 + 에폭 30 + paraphrase
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905_over.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 양파 절임 은 너 무오 래 돼 진물 나온 양 파마 냥 냄새 나고 먹자마자 깜짝 놀라 바로 뱉었네요 국물 은 고기 담갔다 뺀 물 마냥 너무 아무 맛 이 안나
답변    : 실망 시켜 드려서 죄송합니다 해당 부분 은 매장 에 명백한 실수 계기 로 더 메뉴 에 신경 쓰겠습니다 죄송합니다 고객 님
예측답변: 고객 여러분의 소중한 후기가 저희 가게에 많은 도움이 되었으면 합니다.
BLEU점수: 0.18154438409756285
리뷰    : 야무지게 포장 해주셔서 잘 먹고있어요 감사합니다
답변    : 시간 이 지났지만 기억나네요 리뷰 감사합니다 건강 하시고 행복 하세요
예측답변: 저희가 초심을 잃지 않고 최선을 다한다는 것을 고객님의 소중한 평가에 힘입어 보여드리겠습니다.
BLEU점수: 0.19130757147472308
리뷰    : 양도 많고 엄청 맛있어요
답변    : 합리 적 인 가격 에 대 용량 음료 를 제공 해드리고자 열심히 노력 하고 있답니다 음료 부터 디저트 까지 알차게 즐겨주신 것 같아 넘 넘 행복해요 언제나 만족스러운 순간 이 되시길
예측답변: 푸 짐하구 맛있는 맛있게 잘 먹어 주신것 같아 기분 좋 습 니 다. 맛 과 양으로 고객 님들께 보답 하는 되도록 노력 하겠습니다 감사
BLEU점수: 0.11416994971321416
리뷰    : 라멘 은 매 장가 서 먹어야 맛 날거 같아요 맛 이 없는건 아닌데 뭔가 부족한 느낌 아무튼 잘 먹었어요
답변    : 맞아요 아무래도 바로 드시는게 더 맛있을거 같아요 감사합니다
예측답변: 고객 님의 소중한 평가 덕분에 저희 음식이 이렇게 잘 될 수 있었습니다 감사합니다.
BLEU점수: 0.05914474481860569
리뷰    : 생일 이라 시켰는데 너무 맛있게 잘 먹었어요 잡채 랑 미역국 이 특히 맛있고 고기 는 살짝 퍽퍽 하긴 했지만 전체 적 으로 다 너무 맛있었 습 니당
답변    : 안녕하세요 석쇠 불고기 와 입맛 에 맞으셨다니 정말 기쁘 네 용 1년 365일 생일상 처럼 드셨으면 하는

In [ ]:
# 0905 데이터 + 긍부정 비율 2:1 + 에폭 30 + paraphrase
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 갈비 진짜 너무 맛있네용 반신반의 하면서 시 켯 는데 갈비 맛 나구 진짜 맛 잇 어유 치킨 도 구 웃 잘 먹었 슴 당
답변    : 안녕하세요 맛있게 드셔주셔서 감사합니다 치킨 조합 정말 별미 고객 님 께서 치킨 이 땡기실 때 마다 저희 가 생각나도록 더욱 열심히 하겠습니다 아 주문 정말 감사 드리고 다음 에 또 봐요


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


예측답변: 저희 음식이 입맛에 맞았다면 지금처럼 힘든 시기는 없었을 것이라 생각하는데 고객 여러분의 소중한 후기 덕분입니다 감사합니다.
BLEU점수: 0.2696986267476108
리뷰    : 맛있는 냄새 에 잘 먹을게요 비 조리 인데 아쉽게 리뷰 이벤트 안 적어서 인지 사리 가 없네요
답변    : 에 이 쿵 아쉬워서 어 쩌지 요 말 씀이 없으시면 챙겨 드리 기 가 힘듭니다 다음 에는 꼭 적어 주세요 더 맛있게 드실 수 있게 같이 드릴게요
예측답변: 저희가 이 부분은 운영을 이쪽에서 하기 때문에 고객님들이 불편을 많이 느끼셨겠습니다만 양해 부탁드립니다.
BLEU점수: 3.176496675533262e-78
리뷰    : 옆집 문 앞 에 배달 해주셨네요 맛있어서 다 먹고 사짘 찍어 버렸어요 잘 먹었습니다
답변    : 앗 죄송합니다 고객 님 그래도 너그러이 이해 해주시고 리뷰 까지 남겨주셔서 너무 감사해요 더 노력 하는 되겠습니다 늘 행복하세용


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


예측답변: 안녕하세요 고객 님 다음에도 더 좋은 서비스와 맛있는 요리로 찾아 뵙겠습니다.
BLEU점수: 0.24830724520513484
리뷰    : 몸 이 안좋아서 시켜 봤는데 양도 많고 맛 도 좋아요
답변    : 리뷰 남겨주셔서 감사합니다 칭찬 해 주시 니 힘 이 팍팍 나네요 저희 음식 이 기력 을 회복하시는데 도움 이 되기만 한다면 그것 만으로도 너무나 행복합니다 발전 하는 맛 과 좋은 서비스
예측답변: 고객 여러분 감사합니다 이렇게 좋은 요리 만들어 주셔서 요즘 같이 힘든 시기에 정말 감사드립니다.
BLEU점수: 0.22491637465159936
리뷰    : 먹고나면 속이 편하고 확실히 맛있어요 가라 아케 도 오랜 만 에 시켜 먹었는데 잘 먹었습니다
답변    : 반갑습니다 오늘 도 맛있게 드시고 예쁜 사진 리뷰 정말 감사 드립니다
예측답변: 고객 님 맛있게 먹어주셔서 감사합니다.
BLEU점수: 0.19154624426653655
리뷰    : 맛 도 좋고 배달 도 빨 라서 따뜻하게 맛있게 먹었어요
답변    : 즐거운 시간 보내시고 소중한 리뷰 작성 해주셔서 감사 드려요 메뉴 별로 가장 맛있는 온도 가 있다고 해 요 많은 분 들 이 매장 에서 찾아주시는 이유 이기도 하죠 저희 도 이러한
예측답변: 당일 삶은 보쌈 직접 담근 양념과 신선한 음식 재료로 고객 밥상에 항상 오르는 것이 바로 이 요리입니다 맛있게 먹어 주셔서 감사합니다
BLEU점수: 0.07196816797546395
리뷰    : 폴트 버거 너무 맛있고 감자 튀김 도 정말 맛있네요 배달 도 빨리 해주셔서 따뜩하 게 잘 먹었어요 포장 도 너무 예쁘고 테니스 컨셉 매장 도 꼭 한번 가보고싶어요
답변    : 소중한 리뷰 글 감사 드립니다 저희 항상 고객 님 을 향 한 마음 으로 맛있는 수제 찾아 뵙도록 노력 하겠습니다 감사합니다
예측답변: 당일 삶은 삼겹살과 직접 담근 김치, 그리고 신선한 식자재를 가지고 고객 밥상에 매번 정성을 다하는 요리 연구가 계속 되고 있습니다.
BLEU점수:

In [ ]:
# 0905 데이터 + 긍부정 비율 5:5 + 에폭 30
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 간장 은 묽어서 좀 아쉬운데 나머진 맛 잇어요
답변    : 고객 님 안녕하세요 골라 먹는 부천대 주문 에 맛있게 드셔 주셔서 정말 감사합니다 맛있게 드셨다는 칭찬 은 언제 들어도 참 기분 이 좋아지는 그런 말 인거 같아요 덕분 에 힘든 시기
예측답변: 고객 님 소중한 리뷰 감사합니다 저희는 저희 민의의 고기를 두 툼 하 게 썰고 있습니다 광어 도미 등 이 이 이 저희 매장으로 연락 주세요 묻고 따지지
BLEU점수: 0.1506279696450486
리뷰    : 오 여기 좋네요 시켜놓고 낮잠 자 다가 늦게 먹었긴 했는데 진짜 빨리 와요
답변    : 감사합니다 주문 즉시 최대한 빨리 조리 해서 보내 드리고있어요 빠른 베차 시 스팀 이기 에 고객 님 들이 우선 적 으로 잡아 가는 편이 에요
예측답변: 안녕하세요 고객 님 소중한 리뷰 감사합니다 주셔서 감사합니다 감사드립니다 항상 고객 님의 소리는 저희를 춤추게 합니다 앞으로 계속해서 정성스러운 음식을 제공해 드리도록 약속
BLEU점수: 0.18665030216111525
리뷰    : 닭발 좋아해서 여기저기 주문 해서 먹는데 공주 닭발 맛있습니다 배달 도 무지 빠르고 센스 있는 서비스 까지 감사합니다 맛있게 잘 먹었습니다
답변    : 항상 같은 맛 유지 하도록 노력 할게요 항상 맛 과 청결 에 정성 을 다 하겠습니다 주문 해주셔서 감사합니다
예측답변: 고객 님 소중한 리뷰 감사합니다 감사드립니다 고객 님의 만족을 위해 노력하는 되겠습니다 즐거운 시간 보내시고 행복한 나날 되세요
BLEU점수: 0.18289441873227932
리뷰    : 최고 너무 맛있고 배달 도 빠르고 잘 먹었습니다
답변    : 반갑습니다 맛있게 드셔주시고 잊지 않고 리뷰 남겨 주심 에 큰 힘 을 얻습니다 늘 변함 없는 맛 과 친절한 서비스 로 보답 하겠습니다 항상 행복하세요
예측답변: 안녕하세요 고객 님 저희 매장으로 찾아뵙겠습니다
BLEU점수: 0.0218228208440802
리뷰    : 오늘 은 주 문 한 거 빠

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


예측답변: 주문해 주셔서 감사합니다 드립니다 다음에 또 주문해 주세요
BLEU점수: 0.06122493138554343
리뷰    : 전체 적 으로 맛있네요 하와 이안 에 코코넛 은 좀 그렇긴 한데 요 기 서 만 계속 피자 먹네요
답변    : 맛있게 드셔주시고 소중한 리뷰 달아주셔서 정말 감사 드립니다
예측답변: 안녕하세요 고객 님 저희 매장을 찾아주셔서 너무 감사드립니다 앞으로도 저희 음식을 이용해 주셔서 정말 감사드립니다 다음에 또 주문해 주세요
BLEU점수: 0.18463832893957804
리뷰    : 사장 님 맛있게 잘 먹었습니다 번창 하세요
답변    : 안녕하세요 고객 님 맛있게 드시고 소중한 리뷰 와 별 5 개 감사합니다 언제 주문 하셔도 푸 짐 하고 맛있는 닭발 을 위해 항상 최선 을 다 하겠습니다 앞 으로도 많은 사랑
예측답변: 안녕하세요 고객 님 맛있게 드시고 소중한 리뷰 남겨주셔서 정말 감사합니다 드립니다 고객 님의 소리는 저희를 춤추게 합니다 앞으로 계속해서 정성스러운 음식을 제공해 드리도록 약속
BLEU점수: 0.4093854533293307


In [ ]:
# yogiyo 전체데이터 모델링
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 먹을만 합니다 추천 해 요
답변    : 좋은 리뷰 감사 드립니다 저희 은 리뷰 평점 이나 표면 적 으로만 좋은 집 이 아닌 드셧 던 분들 이 믿고 다시 드시고싶은 집 이 되자 라는 마음 으로 모두 가
예측답변: 고객 님 소중한 고객 네 이 이임 이임 맛있게 드셨다니 너 어어 어 무 기쁜 네 용 만족스러운 식사하셨다면에 용 만족스러운 식사하셨다면 찜 가
BLEU점수: 1.7456357302038282e-78
리뷰    : 여친 이 너무 맛있게 잘 먹었답니다 감사합니다 사장 님 번창 하세요
답변    : 안녕하세요 토핑 을 아끼지 않는 미 미 주문 주시 고 리뷰 로 찾아와 주셔서 정말 감사 드립니다 앞 으로도 맛있는 생각나실 때 저희 미 또 찾아주세요 감사합니다
예측답변: 맛있게 드셔주셔서 감사합니다 요청 사항 은 조 리전 늘 꼼꼼하게 체크하고 있으니 필요하신 사항 있으시면 다음 주 문 시 주 문 메모에 남겨주세요 최대한 반영해서 보내 드리겠습니다 믿고 주문해
BLEU점수: 0.1556563403952443
리뷰    : 주말 의 마무리 는 역시
답변    : 고객 님 리뷰 에 몸 이 절로 들썩입니다 시간 날 때 마다 리뷰 훑어 보며 반응 살피 고 있는데 덕분 에 넘 힘 이나고 힐링 이 되네요 리뷰 너무 감사 드립니다
예측답변: 안녕하세요 고객 님 소중한 리뷰 남겨주셔서 감사합니다 저희 큰아들 백 순대는 항상 맛있는 서비스를 해드리기 위해 서 노력하고 있습니다 사용되는 모든 곱창 은 국내 산 제 주산
BLEU점수: 0.12288378213016357
리뷰    : 고기 가 맛있어요 사장 님 이 서비스 로 쌈장 도 주셨어요 배달 도 예상 시간 보다 빠흐 게 옵니다
답변    : 저희 목살 만족스럽고 맛있게 잘 드셨다니 기쁩니다 소중한 리뷰 와 저희 에게 큰 힘 을 주는 좋은 평가 남겨 주셔서 정말 감사 드립니다 무더운 날씨 에 건강 잘 지키시고 오늘 도
예측답변: 맛있게 드셔 주시 고 소중한 리뷰 남겨주셔서 감사합니다 만족스러운 식사하신 것 같아 오

In [ ]:
# yogiyo 전체데이터 모델링
STATEDICT_OV_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase_clean.pt'
model.load_state_dict(torch.load(STATEDICT_OV_PATH))
random_evaluation_10(model, test_dataset, train_dataset.wordvocab.index2word, device)

리뷰    : 주문 에 후라이드 잇 슈가 있었지만 다 맛있었습니다
답변    : 저희 을 항상 찾아주시고 성원 해주시는 분 들 께 감사 의 뜻 을 전하 고 싶을 만큼 귀한 시간 내어 주셔서 감사합니다 .
예측답변: 항상 저희 직구 삼이 직구 삼이 되도록 노력 직구 삼이 되도록 노력 하겠습니다 감사합니다 .
BLEU점수: 0.14828218247800706
리뷰    : 맛있습니다 배달 도 빠르고 양도 많아오
답변    : 항상 건강하고 행복한 나날 들 이 되시길 기원 합니다 . 고객 님 들의 소중한 정보 감사 드립니다 .
예측답변: 항상 건강하고 행복한 나날 들 이 되시길 기원 합니다 .
BLEU점수: 0.4464393987758162
리뷰    : 디저트 하나 하나 다 맛있어요 휘 낭시 에 랑 소금 빵 완전 맛있어요 이 즈 니 버터 프레첼 은 팥 앙금 들어있는건 줄 알았는데 다시 보니 아니더라 규 요 그치만 이
답변    : 이 즈 니 버터 프리첼 의 팥 앙금 버전 앙 버터 도 뚜레쥬르 팀 에 제안 해 보겠습니다 .
예측답변: 이 즈 니 버터 프리첼 의 팥 앙금 버전 앙 버터 도 뚜레쥬르 팀 에 제안 해 보겠습니다 .
BLEU점수: 1.0
리뷰    : 국물 이 깔끔하고 맛있어요 또 먹고 싶은 맛 이네 요
답변    : 저희 국밥 을 많이 사랑 해 주세요 :) 항상 최선 을 다해 준비 하는 만큼 더 좋은 맛 과 영양 으로 보답 하겠습니다 .
예측답변: 저희 국밥 을 많이 사랑 해 주세요 :) 항상 최선 을 다해 준비 하는 만큼 더 좋은 맛 과 영양 으로 보답 하겠습니다 .
BLEU점수: 1.0
리뷰    : 커피 도 부드럽고 다 맛 나네요 잘 먹었습니다
답변    : 항상 정갈한 브런치 맛있는 브런치 준비 하고 고객 님 기다릴게요 곧 또 만나요 그때 까지 행복한 시간 되세요 .
예측답변: 항상 정갈한 브런치 는 브런치 준비 가 준비 기다릴게요 기다릴게요 곧 또 찾아주세요 행복한 하루 되세요 .
BLEU점수: 0.5833316665807692
리뷰    : 맛